In [77]:
import pandas as pd
import datetime
import re
import yfinance as yf
from io import StringIO


In [93]:
table_txt = """
Nombre	Variación diaria	Último Operado AR$	Cantidad	Tenencia US$
AR$	0.00%	1,00	10.725,15	0,08 %
US$ MEP	0.00%	1.022,22	5.230,32	39,90 %
SPY	-1.30%	23.376,50	177,00	30,88 %
AAPL	-2.35%	19.633,50	84,00	12,31 %
DIA	-1.79%	18.585,00	54,00	7,49 %
AMZN	-0.63%	1.030,00	885,00	6,80 %
NVDA	+0.11%	20.376,50	6,00	0,91 %
CVX	-2.72%	18.486,00	2,00	0,28 %
AL30	-2.76%	38.868,50	255,00	0,74 %
GD30	-4.07%	41.200,00	200,00	0,61 %
"""

In [94]:
data = StringIO(table_txt)
df = pd.read_csv(data, delimiter='\t')
df

,SPY,Compra,"US$ 23,40",54,"US$ 0,00",48hs,Cancelada
0,DIA,Compra,"US$ 18,60",11,"US$ 204,60",48hs,Ejecutada
1,AAPL,Compra,"US$ 19,65",4,"US$ 78,60",48hs,Ejecutada
2,AAPL,Venta,"US$ 19,80",60,"US$ 1.188,00",48hs,Ejecutada
3,MRCQO,Venta,"US$ 98,00",31,"US$ 30,38",48hs,Ejecutada
4,AAPL,Compra,"US$ 19,65",90,"US$ 1.768,50",48hs,Ejecutada
...,...,...,...,...,...,...,...
68,SPY,Venta,"US$ 23,70",64,"US$ 1.516,80",48hs,Ejecutada
69,SPY,Compra,"US$ 23,30",36,"US$ 838,80",48hs,Ejecutada
70,SPY,Compra,"US$ 23,30",33,"US$ 768,90",CI,Ejecutada
71,SPY,Venta,"US$ 23,85",251,"US$ 0,00",48hs,Cancelada


In [95]:
def correct_str(input_str):
    input_str = input_str.replace(".", "")
    input_str = input_str.replace(",", ".")

    input_str = re.sub("[^0-9^.]", "", input_str)

    if input_str == '':
        input_str = 0.0
    return float(input_str)

In [81]:
for col in df.columns:
    if col != "Nombre":
        df[col] = df[col].apply(correct_str)

In [82]:
cedear_data = pd.read_excel("data/cedear_ratios_reloaded.xlsx", sheet_name=0)

In [83]:
df2 = df.merge(cedear_data, right_on="symbol", left_on="Nombre", how="left")

In [84]:
def find_price(stock_name):
    if stock_name is None:
        return 0.0
    else:
        return yf.download(stock_name, period="1d", interval="1d")["Adj Close"].iloc[0]

In [85]:
df2["base_symbol"] = df2["base_symbol"].astype(str)

In [86]:
df2["base_asset_price"] = df2["base_symbol"].apply(find_price)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Nombre,Variación diaria,Último Operado AR$,Cantidad,Tenencia US$,symbol,base_symbol,ratio,aux,type,currency,base_asset_price
0,AR$,0.0,1.00,10725.15,0.08,NaN,nan,NaN,NaN,NaN,NaN,10.835000
1,US$ MEP,0.0,1022.22,5230.32,39.90,NaN,nan,NaN,NaN,NaN,NaN,10.835000
2,SPY,130.0,23376.50,177.00,30.88,SPY,SPY,20.0,Y,base,ARS,469.295013
3,AAPL,235.0,19633.50,84.00,12.31,AAPL,AAPL,10.0,L,base,ARS,198.110001
4,DIA,179.0,18585.00,54.00,7.49,DIA,DIA,20.0,A,base,ARS,372.279999
5,AMZN,63.0,1030.00,885.00,6.80,AMZN,AMZN,144.0,N,base,ARS,150.066299
6,NVDA,11.0,20376.50,6.00,0.91,NVDA,NVDA,24.0,A,base,ARS,490.100006
7,CVX,272.0,18486.00,2.00,0.28,CVX,CVX,8.0,X,base,ARS,148.559906
8,AL30,276.0,38868.50,255.00,0.74,NaN,nan,NaN,NaN,NaN,NaN,10.835000
9,GD30,407.0,41200.00,200.00,0.61,NaN,nan,NaN,NaN,NaN,NaN,10.835000


In [87]:
df2["this_asset_price"] = df2["base_asset_price"] / df2["ratio"]

In [88]:
df2.set_index("Nombre", inplace=True)

In [89]:
today_str = datetime.date.today().strftime("%m/%d/%Y")
ccl_dollar = pd.read_csv("data/dolar_ccl_historic.csv", index_col=0)

In [90]:
df2.loc["AR$","this_asset_price"] = 1 / ccl_dollar.loc[today_str, "Referencia"]
df2.loc["US$ MEP","this_asset_price"] = 1


In [91]:
for index, row in df2.iterrows():
    if pd.isna(row["this_asset_price"]):
        stock_name = index + "D.BA"
        try:
            price = yf.download(stock_name, period="1d", interval="1d")["Adj Close"].iloc[0]
            df2.loc[index,"this_asset_price"] = price
        except:
            pass


[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['AL30D.BA']: Exception('%ticker%: No data found, symbol may be delisted')


[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['GD30D.BA']: Exception('%ticker%: No data found, symbol may be delisted')


In [92]:
df2[["Cantidad", "this_asset_price"]]


,Cantidad,this_asset_price
Nombre,,
AR$,10725.15,0.001005
US$ MEP,5230.32,1.000000
SPY,177.00,23.464751
AAPL,84.00,19.811000
DIA,54.00,18.614000
AMZN,885.00,1.042127
NVDA,6.00,20.420834
CVX,2.00,18.569988
AL30,255.00,NaN
